In [1]:
import torch
import torch_pruning as tp
from yolox.models import create_yolox_model

model = create_yolox_model(num_classes=20,name = 'yolox_custom',  
exp_path = '/workspace/tensorrt/YOLOX/exps/default/custom/original/yolox_voc_tiny.py',
ckpt_path = '/workspace/tensorrt/YOLOX/fpgm_1/best_ckpt.pth'
).cuda()
# 1. setup strategy (L1 Norm)
strategy = tp.strategy.L1Strategy() # or tp.strategy.RandomStrategy()

/usr/local/lib/python3.8/dist-packages/tqdm-4.64.0-py3.8.egg/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# 2. build dependency graph for resnet18
DG = tp.DependencyGraph()
DG.build_dependency(model, example_inputs=torch.randn(1,3,224,224).cuda())



In [3]:

temp = []
temp2 = list(model.state_dict().keys())
num_list = [0,1,2,3]
#print(temp2)
for i in range(len(temp2)):
        if 'bn' in temp2[i]:
            continue
        if 'weight' in temp2[i]:
            temp2[i] = temp2[i].replace('.weight','')
            temp2[i] = temp2[i].replace('2.0','2[0]')
            temp2[i] = temp2[i].replace('2.1','2[1]')
            temp2[i] = temp2[i].replace('3.0','3[0]')
            temp2[i] = temp2[i].replace('3.1','3[1]')
            temp2[i] = temp2[i].replace('4.0','4[0]')
            temp2[i] = temp2[i].replace('4.1','4[1]')
            temp2[i] = temp2[i].replace('5.0','5[0]')
            temp2[i] = temp2[i].replace('5.1','5[1]')
            temp2[i] = temp2[i].replace('5.2','5[2]')
            temp2[i] = temp2[i].replace('m.0','m[0]')
            temp2[i] = temp2[i].replace('m.1','m[1]')
            temp2[i] = temp2[i].replace('m.2','m[2]')
            temp.append(temp2[i])

print(temp)
# backbone 만 프루닝


['backbone.backbone.stem.conv.conv', 'backbone.backbone.dark2[0].conv', 'backbone.backbone.dark2[1].conv1.conv', 'backbone.backbone.dark2[1].conv2.conv', 'backbone.backbone.dark2[1].conv3.conv', 'backbone.backbone.dark2[1].m[0].conv1.conv', 'backbone.backbone.dark2[1].m[0].conv2.conv', 'backbone.backbone.dark3[0].conv', 'backbone.backbone.dark3[1].conv1.conv', 'backbone.backbone.dark3[1].conv2.conv', 'backbone.backbone.dark3[1].conv3.conv', 'backbone.backbone.dark3[1].m[0].conv1.conv', 'backbone.backbone.dark3[1].m[0].conv2.conv', 'backbone.backbone.dark3[1].m[1].conv1.conv', 'backbone.backbone.dark3[1].m[1].conv2.conv', 'backbone.backbone.dark3[1].m[2].conv1.conv', 'backbone.backbone.dark3[1].m[2].conv2.conv', 'backbone.backbone.dark4[0].conv', 'backbone.backbone.dark4[1].conv1.conv', 'backbone.backbone.dark4[1].conv2.conv', 'backbone.backbone.dark4[1].conv3.conv', 'backbone.backbone.dark4[1].m[0].conv1.conv', 'backbone.backbone.dark4[1].m[0].conv2.conv', 'backbone.backbone.dark4[1].m

In [4]:
print(model)

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(48, 24, kernel_siz

In [5]:
from pruning_imagenet import Mask

In [6]:
m = Mask(model)

In [7]:
m.init_length()

In [8]:
m.init_mask(1, 0.25)
print(m.mask_index)

filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
mask Ready
[24, 27, 30, 33, 39, 45, 54, 57, 60, 63, 69, 75, 90, 93, 99, 102, 96]


In [9]:
m.do_mask()
m.do_similar_mask()

mask Done
mask similar Done


In [10]:
m.if_zero()
print(m.mask_index)
print(m.similar_matrix)

layer: 24, number of nonzero weight is 3456, zero is 1152
layer: 27, number of nonzero weight is 3456, zero is 1152
layer: 30, number of nonzero weight is 6912, zero is 2304
layer: 33, number of nonzero weight is 1728, zero is 576
layer: 39, number of nonzero weight is 1728, zero is 576
layer: 45, number of nonzero weight is 1728, zero is 576
layer: 54, number of nonzero weight is 13824, zero is 4608
layer: 57, number of nonzero weight is 13824, zero is 4608
layer: 60, number of nonzero weight is 27648, zero is 9216
layer: 63, number of nonzero weight is 6912, zero is 2304
layer: 69, number of nonzero weight is 6912, zero is 2304
layer: 75, number of nonzero weight is 6912, zero is 2304
layer: 90, number of nonzero weight is 55296, zero is 18432
layer: 93, number of nonzero weight is 55296, zero is 18432
layer: 96, number of nonzero weight is 110592, zero is 36864
layer: 99, number of nonzero weight is 27648, zero is 9216
layer: 102, number of nonzero weight is 248832, zero is 82944
[2

In [11]:
model = m.model


In [12]:
dict ={}


for key, value in model.state_dict().items():
    temp = []
    if len(value.shape) ==4:
        print('pruning channel and layer name : ',len(value),',', key)
        for i in range(len(value)):
            if 'conv3' in key:
                continue
            if value[i].sum() == 0: 
                print('pruning channex index : ', i)
                #temp = torch.norm(value.view(len(value), -1) , p= 1, dim=1)
                #print(temp)
                #dict[key].append(i)
                temp.append(i)
    if temp:   
        dict[key]  = temp

pruning channel and layer name :  24 , backbone.backbone.stem.conv.conv.weight
pruning channel and layer name :  48 , backbone.backbone.dark2.0.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.conv1.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.conv2.conv.weight
pruning channel and layer name :  48 , backbone.backbone.dark2.1.conv3.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.m.0.conv1.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.m.0.conv2.conv.weight
pruning channel and layer name :  96 , backbone.backbone.dark3.0.conv.weight
pruning channel and layer name :  48 , backbone.backbone.dark3.1.conv1.conv.weight
pruning channex index :  0
pruning channex index :  22
pruning channex index :  23
pruning channex index :  24
pruning channex index :  25
pruning channex index :  26
pruning channex index :  30
pruning channex index :  31
pruning channex index :  33
pruning ch

In [13]:
print(dict)
#del dict['backbone.backbone.dark3.1.conv3.conv.weight'][0]

{'backbone.backbone.dark3.1.conv1.conv.weight': [0, 22, 23, 24, 25, 26, 30, 31, 33, 37, 41, 42], 'backbone.backbone.dark3.1.conv2.conv.weight': [6, 20, 26, 29, 30, 31, 32, 34, 37, 38, 41, 46], 'backbone.backbone.dark3.1.m.0.conv1.conv.weight': [4, 11, 14, 18, 21, 27, 36, 37, 38, 40, 41, 43], 'backbone.backbone.dark3.1.m.1.conv1.conv.weight': [1, 11, 12, 16, 18, 20, 23, 24, 26, 33, 38, 42], 'backbone.backbone.dark3.1.m.2.conv1.conv.weight': [2, 12, 15, 16, 17, 27, 29, 30, 33, 34, 36, 40], 'backbone.backbone.dark4.1.conv1.conv.weight': [4, 9, 17, 19, 23, 24, 25, 26, 31, 37, 46, 49, 59, 63, 64, 66, 68, 72, 75, 76, 80, 84, 85, 89], 'backbone.backbone.dark4.1.conv2.conv.weight': [1, 2, 9, 14, 19, 26, 35, 39, 42, 43, 45, 48, 52, 56, 63, 65, 72, 74, 75, 84, 85, 86, 92, 94], 'backbone.backbone.dark4.1.m.0.conv1.conv.weight': [0, 20, 22, 36, 37, 39, 40, 41, 44, 47, 49, 53, 54, 60, 61, 62, 63, 64, 71, 72, 73, 75, 91, 93], 'backbone.backbone.dark4.1.m.1.conv1.conv.weight': [15, 18, 27, 29, 33, 40

In [14]:

temp = []
temp2 = list(dict.keys())
for i in range(len(temp2)):
        if 'bn' in temp2[i]:
            continue
        if 'weight' in temp2[i]:
            temp2[i] = temp2[i].replace('.weight','')
            temp2[i] = temp2[i].replace('2.0','2[0]')
            temp2[i] = temp2[i].replace('2.1','2[1]')
            temp2[i] = temp2[i].replace('3.0','3[0]')
            temp2[i] = temp2[i].replace('3.1','3[1]')
            temp2[i] = temp2[i].replace('4.0','4[0]')
            temp2[i] = temp2[i].replace('4.1','4[1]')
            temp2[i] = temp2[i].replace('5.0','5[0]')
            temp2[i] = temp2[i].replace('5.1','5[1]')
            temp2[i] = temp2[i].replace('5.2','5[2]')
            temp2[i] = temp2[i].replace('m.0','m[0]')
            temp2[i] = temp2[i].replace('m.1','m[1]')
            temp2[i] = temp2[i].replace('m.2','m[2]')
            temp.append(temp2[i])


import copy
print(temp)
key = []
temp2 = copy.deepcopy(temp)
for i in range(len(temp2)):
      
            temp2[i] = temp2[i].replace('[0][0]','.0.0')
            temp2[i] = temp2[i].replace('[0][1]','.0.1',)
            temp2[i] = temp2[i].replace('[1][0]','.1.0',)
            temp2[i] = temp2[i].replace('[1][1]','.1.1',)
            temp2[i] = temp2[i].replace('[2][0]','.2.0',)
            temp2[i] = temp2[i].replace('[2][1]','.2.1',)
            temp2[i] = temp2[i].replace('3[0]','3.0',)
            temp2[i] = temp2[i].replace('3[1]','3.1')
            temp2[i] = temp2[i].replace('4[0]','4.0',)
            temp2[i] = temp2[i].replace('4[1]','4.1',)
            temp2[i] = temp2[i].replace('5[0]','5.0',)
            temp2[i] = temp2[i].replace('5[1]','5.1',)
            temp2[i] = temp2[i].replace('5[2]','5.2',)
            temp2[i] = temp2[i].replace('m[0]','m.0',)
            temp2[i] = temp2[i].replace('m[1]','m.1',)
            temp2[i] = temp2[i].replace('m[2]','m.2',)
            temp2[i] = temp2[i].replace('[0]','.0',)
            temp2[i] = temp2[i].replace('[1]','.1',)
            temp2[i] = temp2[i].replace('[2]','.2',)
            key.append(temp2[i])

print(key)
print(temp)

['backbone.backbone.dark3[1].conv1.conv', 'backbone.backbone.dark3[1].conv2.conv', 'backbone.backbone.dark3[1].m[0].conv1.conv', 'backbone.backbone.dark3[1].m[1].conv1.conv', 'backbone.backbone.dark3[1].m[2].conv1.conv', 'backbone.backbone.dark4[1].conv1.conv', 'backbone.backbone.dark4[1].conv2.conv', 'backbone.backbone.dark4[1].m[0].conv1.conv', 'backbone.backbone.dark4[1].m[1].conv1.conv', 'backbone.backbone.dark4[1].m[2].conv1.conv', 'backbone.backbone.dark5[2].conv1.conv', 'backbone.backbone.dark5[2].conv2.conv', 'backbone.backbone.dark5[2].m[0].conv1.conv', 'backbone.backbone.dark5[2].m[0].conv2.conv']


In [15]:
value = list(dict.values())
value

[[0, 22, 23, 24, 25, 26, 30, 31, 33, 37, 41, 42],
 [6, 20, 26, 29, 30, 31, 32, 34, 37, 38, 41, 46],
 [4, 11, 14, 18, 21, 27, 36, 37, 38, 40, 41, 43],
 [1, 11, 12, 16, 18, 20, 23, 24, 26, 33, 38, 42],
 [2, 12, 15, 16, 17, 27, 29, 30, 33, 34, 36, 40],
 [4,
  9,
  17,
  19,
  23,
  24,
  25,
  26,
  31,
  37,
  46,
  49,
  59,
  63,
  64,
  66,
  68,
  72,
  75,
  76,
  80,
  84,
  85,
  89],
 [1,
  2,
  9,
  14,
  19,
  26,
  35,
  39,
  42,
  43,
  45,
  48,
  52,
  56,
  63,
  65,
  72,
  74,
  75,
  84,
  85,
  86,
  92,
  94],
 [0,
  20,
  22,
  36,
  37,
  39,
  40,
  41,
  44,
  47,
  49,
  53,
  54,
  60,
  61,
  62,
  63,
  64,
  71,
  72,
  73,
  75,
  91,
  93],
 [15,
  18,
  27,
  29,
  33,
  40,
  41,
  43,
  44,
  47,
  48,
  50,
  53,
  54,
  55,
  56,
  58,
  60,
  68,
  84,
  86,
  93,
  94,
  95],
 [6,
  11,
  16,
  17,
  18,
  20,
  23,
  27,
  28,
  33,
  35,
  38,
  40,
  47,
  50,
  55,
  67,
  71,
  78,
  84,
  88,
  89,
  90,
  92],
 [1,
  6,
  10,
  11,
  14,
  16

In [16]:
for i in range(len(temp)):
    #pruning_plan = DG.get_pruning_plan(eval(f"model.{temp[i]}"), tp.prune_conv_out_channel, value[i])
    pruning_plan = DG.get_pruning_plan(eval(f"model.{temp[i]}"), tp.prune_conv_out_channel, dict[(key[i]) + '.weight'])
    print(pruning_plan)
    if DG.check_pruning_plan(pruning_plan):
        pruning_plan.exec()     

Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)

--------------------------------
          Pruning Plan
--------------------------------
User pruning:
[ [DEP] ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)) => ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)), Index=[0, 22, 23, 24, 25, 26, 30, 31, 33, 37, 41, 42], metric={'#params': 1152}]

Coupled pruning:
[ [DEP] ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)) => BatchnormPruner on backbone.backbone.dark3.1.conv1.bn (BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)), Index=[0, 22, 23, 24, 25, 26, 30, 31, 33, 37, 41, 42], metric={'#params': 24}]
[ [DEP] BatchnormPruner on backbone.backbone.dark3.1.conv1.bn (BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, tr

In [17]:
print(model)

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(48, 24, kernel_siz

In [18]:
ckpt_state = {
                "model": model.state_dict(),
            }

torch.save(ckpt_state, 'type3_0.25_b_v1.pth')


